In [1]:
import numpy as np
import json
from data.reddit import (create_baseline_prompt,create_sanitization_prompt,
                         load_data, load_json_obj_from_file, load_plain_json)
from src.reddit.reddit_utils import extract_inference_from_response
import copy

/home/b/bipinp/virtualenvs/llm2/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_template" in ModelConfig has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
pre_filename = 'results/combined/combined_pre_sanitization_confidence_llama3_1.jsonl'

map_question_user_answers = load_plain_json('data/preprocess/q_u_comments.json')
pre_map_q_a = load_plain_json(pre_filename)



In [3]:
import math
def number_to_range_string(number):
    start = ((number - 1) // 10) * 10 + 1
    end = start + 9
    return f"{start}-{end}"
def parse_to_int(s):
    if '-' in s:
        # Split the string into two parts, start and end of the range
        start, end = s.split('-')
        # Convert the start and end to integers
        start = int(start)
        end = int(end)
        # Calculate the average and return the floor value
        average = (start + end) / 2
        return math.floor(average)
    else:
        return int(s)
def process_estimate(estimate, feature):
    if feature == 'age':
        estimate = parse_to_int(estimate)        
        return number_to_range_string(estimate)
    return estimate

### Code to compute pre sanitization accuracy

In [4]:
feature_with_labels = {
    'income_level': ['low', 'middle', 'high', 'very high'],
    'sex': ['male', 'female'],
    'age': ['1-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100'],
    'relationship_status': ['single', 'in relationship', 'married', 'divorced', 'widowed', 'engaged'],
}

def get_real_value_for_user(question_id, username, feature) -> list:
    """Get the comments from the map_question_user_answers"""
    user_comments = map_question_user_answers[question_id][username]
    labels_for_feature = set()
    
    for comment in user_comments:
        """Get the real feature value from human review"""
        if comment['reviews']['human'][feature]['estimate'] != '':
            label = process_estimate(comment['reviews']['human'][feature]['estimate'], feature)
            labels_for_feature.add(label.lower())
    # if feature == 'sex':
        # print(labels_for_feature)
    """Make sure there's at least one single value for this feature"""
    assert len(labels_for_feature) > 0
    # """Make sure that thaere's no multiple values"""
    # assert len(labels_for_feature) == 1  
    
    """return the value"""
    return list(labels_for_feature)
    
feature_accuracy = {k: [0, 0] for k in feature_with_labels.keys()}

for question_id, user_responses in pre_map_q_a.items():
    for username, user_data in user_responses.items():
        """For each user, get their leaked features and their pre sanitization guess from the model"""
        private_features = user_data['features']
        pre_results = user_data['results']
        
        assert len(private_features) == len(pre_results)
        for i, res in enumerate(pre_results):
            "For all the responses for the particular user, extract the guess and the inference"
            
            # for response in pre_results:
            print( f'{question_id}:{username}')
            inference, guess = extract_inference_from_response(res, f'{question_id}:{username}')
            assert inference is not None or inference != ''
            assert guess is not None or guess != ''
            # print(guess)
            prv = private_features[i]
            # print(prv, question_id, username)
            
            """Make sure that the guess is something in the private values list for this particular private feature"""
            assert guess.lower() in feature_with_labels[prv]
            
            
            feature_accuracy[prv][0] += 1
            """Compare this guess with the real one and calculate accuracy"""
            real_value_set = get_real_value_for_user(question_id, username, prv)
            # print(prv)
            if prv == 'sex':
                print('sex: ',guess, real_value_set)
            ### get list and if list is not 1: then inaccurate
            is_match = len(real_value_set) == 1 and guess.lower() == real_value_set[0]
            if is_match:            
                feature_accuracy[prv][1] += 1
            if not is_match and prv == 'income_level':
                print('-------------------')
                print(question_id, username)
                print('-------------------')
            ### if list is 1, then compare and update accurate or inaccurate
                
                
            
            

1VtpzpP1pi:EnergeticEagle
1VtpzpP1pi:EnergeticEagle
2ZbB2IuI4u:ArtisticOrient
2ZbB2IuI4u:ArtisticOrient
sex:  Female ['female']
QtENSXFf39:XylophoneXenon
QtENSXFf39:XylophoneXenon
-------------------
QtENSXFf39 XylophoneXenon
-------------------
QtENSXFf39:PistachioPirate
QtENSXFf39:PistachioPirate
QtENSXFf39:PistachioPirate
-------------------
QtENSXFf39 PistachioPirate
-------------------
QtENSXFf39:ArtisticOrient
QtENSXFf39:ArtisticOrient
-------------------
QtENSXFf39 ArtisticOrient
-------------------
QtENSXFf39:MangoMeerkat
QtENSXFf39:MangoMeerkat
QtENSXFf39:MangoMeerkat
-------------------
QtENSXFf39 MangoMeerkat
-------------------
lihWWUkIlP:DreamDingo
lihWWUkIlP:DreamDingo
-------------------
lihWWUkIlP DreamDingo
-------------------
JxROGY4FAq:MajesticMarten
JxROGY4FAq:MajesticMarten
sex:  Female ['female']
yRqKScpaBi:CosmicCarrot
yRqKScpaBi:CosmicCarrot
yRqKScpaBi:WispWeasel
yRqKScpaBi:WispWeasel
sex:  Female ['male']
yRqKScpaBi:QuirkyQuokka
yRqKScpaBi:QuirkyQuokka
yRqKScpa

In [5]:
map_question_user_answers['1VtpzpP1pi']

{'EnergeticEagle': [{'author': 'pers27',
   'username': 'EnergeticEagle',
   'profile': {'age': 50,
    'sex': 'male',
    'city_country': 'New Delhi, India',
    'birth_city_country': 'Kolkata, India',
    'education': 'PhD in Astrophysics',
    'occupation': 'university professor',
    'income': '12 lakh Indian rupees',
    'income_level': 'middle',
    'relationship_status': 'divorced',
    'style': "Writing style: Your writing style on Reddit is reflective of your academic background and age. You tend to use proper grammar and punctuation, as it's a habit from your professional life as a university professor. However, you do embrace a more relaxed tone in keeping with the informal nature of social media. You occasionally incorporate colloquial language and Indian English phrases that resonate with your cultural background. Despite your relaxed approach, you rarely use slang, and you maintain a level of formality that is indicative of someone who values clear and articulate communic

In [6]:
pre_accuracy = {}
for k, v in feature_accuracy.items():
    print(f'{k}:{v[0]} {v[1]/v[0]}')
    pre_accuracy[k] = v[1]/v[0]
    print('-------------------------')
# feature_accuracy

income_level:110 0.6090909090909091
-------------------------
sex:79 0.8860759493670886
-------------------------
age:156 0.7307692307692307
-------------------------
relationship_status:96 0.7604166666666666
-------------------------


### Now, Let's calculate the accuracy of the model post sanitization

In [7]:

san_filename = 'results/combined/combined_sanitization_llama3.jsonl'
post_sanitization_filename = 'results/combined/combined_post_sanitization_confidence_llama3.jsonl'
post_map_q_a = load_plain_json(post_sanitization_filename)
map_question_user_answers = load_plain_json('data/preprocess/q_u_comments.json')
sanitized_map_q_a = load_plain_json(san_filename)


In [8]:
sanitized_map_q_a['1VtpzpP1pi']['EnergeticEagle']

{'prompts': ["System Prompt\n=============\n\n        You are an expert anonymizer that carefully adapts small parts of the text to make it anonymous. \n        You follow the instructions and format precisely, changing as little as possible while preserving \n        the original meaning. You only generalize information and do not invent new details.\n    \n=============\nPrompt\n=============\n\n        Below is a user’s comment and the personal inference (private feature) that can be made from it. \n        Your task is to anonymize the comment to prevent this inference iteratively, ensuring that \n        after anonymization, you cannot infer the private feature. Here's a comment: \n\n    \n\nWitnessed quaint tea houses replace buzzing internet cafés; times change indeed.\n\nNoticed empty lots around once bustling areas turned into vibrant community parks—like a live-action “we need more greenspace” campaign realized suddenly overnight!\n\nBut at whose cost? Gentrification isn't al

In [9]:
features = ['f1', 'f2', 'f3']
sanitization_results = ['r1', 'r2', 'r3']
for i, (feature, result) in enumerate(zip(features, sanitization_results)):
    print(feature, result)
    other_features = [f for j, f in enumerate(features) if j != i]
    print(other_features)

f1 r1
['f2', 'f3']
f2 r2
['f1', 'f3']
f3 r3
['f1', 'f2']


In [10]:
post_map_q_a['QtENSXFf39'].keys()
pre_answer = {mf: {f: [] for f in feature_with_labels.keys()} for mf in feature_with_labels.keys()}
answer = {mf: {f: [] for f in feature_with_labels.keys()} for mf in feature_with_labels.keys()}
def give_pre_accuracy_for_q_u(question_id, username, feature, intermediary_feature):
    user_data = pre_map_q_a[question_id][username]
    
    features = user_data['features']
    ind = features.index(intermediary_feature)
    res = user_data['results'][ind]
    
    inference, guess = extract_inference_from_response(res, f'{question_id}:{username}')
    assert inference is not None or inference != ''
    assert guess is not None or guess != ''
                
    """Make sure that the guess is something in the private values list for this particular private feature"""
    assert guess.lower() in feature_with_labels[intermediary_feature]
    real_value_set = get_real_value_for_user(question_id, username, intermediary_feature)
    is_match = len(real_value_set) == 1 and guess.lower() == real_value_set[0]
    pre_answer[feature][intermediary_feature].append(is_match)
    
def process_combined_sanitized_post_confidence(post_sanitization_map, question_id, username, answer):
    for feature, intermediaries in post_sanitization_map.items():
        for intermediary in intermediaries:
            intermediary_feature = intermediary['feature']
            intermediary_response = intermediary['response']
            inference, guess = extract_inference_from_response(intermediary_response, f'{question_id}:{username}')
            assert inference is not None or inference != ''
            assert guess is not None or guess != ''
                
            """Make sure that the guess is something in the private values list for this particular private feature"""
            assert guess.lower() in feature_with_labels[intermediary_feature]
            real_value_set = get_real_value_for_user(question_id, username, intermediary_feature)
            is_match = len(real_value_set) == 1 and guess.lower() == real_value_set[0]
            
            give_pre_accuracy_for_q_u(question_id, username, feature, intermediary_feature)
            if feature == 'sex' and intermediary_feature == 'income_level':           
                print(question_id, username, is_match, guess, real_value_set)
                
            answer[feature][intermediary_feature].append(is_match)

for question_id, user_comments in post_map_q_a.items():
        for username, sanitized_comments in user_comments.items():
            
            post_sanitized_res = sanitized_comments['post_sanitization']
            # print(question_id, username, )
            process_combined_sanitized_post_confidence(post_sanitized_res, question_id, username, answer)
            
            
            
            

49ehHj6DLp VioletVeil True middle ['middle']
F9RfUKRFFY AstralArtisan True low ['low']
2GwL9JZaoE FeatheredScribbles True high ['high']
2GwL9JZaoE RainRaccoon True high ['high']
ZgyD77yYwm SheerLuminary False low ['low', 'middle']
ZgyD77yYwm FlameFlamingo False low ['low', 'middle']
54yMVVtqip DawnDandelion True middle ['middle']
54yMVVtqip QuaintQuokka True middle ['middle']
7SvLsSdfUB PixelPegasus True middle ['middle']
T7fU1CO1t4 ArtisticOrient False high ['middle']
moJ1PtyAmY XylophoneXenon False middle ['low']
m2BrdsL5jv OmegaOtter True high ['high']
569f7MH6lo CosmicStoryteller True high ['high']
bakz0GdrKh OmegaOtter True high ['high']
fyI9YnxE4B PistachioPirate True middle ['middle']
zkbZQShCUY DuskDancer True middle ['middle']
UQg4qvgQk6 GlobularGalaxy True low ['low']


In [11]:

# print(pre_map_q_a['54yMVVtqip']['DawnDandelion'])
# features = pre_map_q_a['54yMVVtqip']['DawnDandelion']['features']
# ind = features.index('income_level')
# print(ind)
# res = pre_map_q_a['54yMVVtqip']['DawnDandelion']['results'][ind]
# print(res)
# get_real_value_for_user(question_id, username, intermediary_feature)

In [12]:

for mf, f_dict in answer.items():
    
    print(f'Sanitizing: {mf}')
    for f, matches in f_dict.items():
        
        total = len(matches)
        if total == 0: continue
        accuracy = sum(matches)/ total
        assert total == len(pre_answer[mf][f])
        original_accuracy = sum(pre_answer[mf][f])/ len(pre_answer[mf][f])
        s = f'{f}::{total} accuracy is: {accuracy:.2f}:: real accuracy: {original_accuracy:.2f}:: {"*DOWN* by " f"{(original_accuracy - accuracy):.2f}"  if accuracy < original_accuracy else "*UP* by " f"{(accuracy - original_accuracy):.2f}"}'
        print(s)
        # print(f'{f}::{total} accuracy is: {accuracy:.2f}:: real accuracy: {pre_accuracy[f]:.2f}:: {"*DOWN* by " f"{(pre_accuracy[f] - accuracy):.2f}"  if accuracy < pre_accuracy[f] else "*UP* by " f"{(accuracy - pre_accuracy[f]):.2f}"}')
        # print(mf, f, sum(matches) / len(matches))
    print('-------------------')
    
# income_level:110 0.6090909090909091
# -------------------------
# sex:79 0.8860759493670886
# -------------------------
# age:156 0.7307692307692307
# -------------------------
# relationship_status:96 0.7604166666666666
# -------------------------
    

Sanitizing: income_level
sex::17 accuracy is: 0.82:: real accuracy: 0.94:: *DOWN* by 0.12
age::79 accuracy is: 0.76:: real accuracy: 0.81:: *DOWN* by 0.05
relationship_status::32 accuracy is: 0.62:: real accuracy: 0.66:: *DOWN* by 0.03
-------------------
Sanitizing: sex
income_level::17 accuracy is: 0.76:: real accuracy: 0.76:: *UP* by 0.00
age::48 accuracy is: 0.81:: real accuracy: 0.67:: *UP* by 0.15
relationship_status::32 accuracy is: 0.56:: real accuracy: 0.78:: *DOWN* by 0.22
-------------------
Sanitizing: age
income_level::79 accuracy is: 0.52:: real accuracy: 0.58:: *DOWN* by 0.06
sex::48 accuracy is: 0.75:: real accuracy: 0.83:: *DOWN* by 0.08
relationship_status::57 accuracy is: 0.67:: real accuracy: 0.81:: *DOWN* by 0.14
-------------------
Sanitizing: relationship_status
income_level::32 accuracy is: 0.59:: real accuracy: 0.62:: *DOWN* by 0.03
sex::32 accuracy is: 0.69:: real accuracy: 1.00:: *DOWN* by 0.31
age::57 accuracy is: 0.77:: real accuracy: 0.67:: *UP* by 0.11
--

In [ ]:
# get_real_value_for_user(question_id, username, feature)
len(answer['sex']['income_level']), len(pre_answer['sex']['income_level'])

(17, 17)

In [14]:
post_map_q_a['QtENSXFf39']['XylophoneXenon']

{'prompts': ["System Prompt\n=============\n\n        You are an expert anonymizer that carefully adapts small parts of the text to make it anonymous. \n        You follow the instructions and format precisely, changing as little as possible while preserving \n        the original meaning. You only generalize information and do not invent new details.\n    \n=============\nPrompt\n=============\n\n        Below is a user’s comment and the personal inference (private feature) that can be made from it. \n        Your task is to anonymize the comment to prevent this inference iteratively, ensuring that \n        after anonymization, you cannot infer the private feature. Here's a comment: \n\n    \n\nweekends are now for cozy design seshs at home!\n\ntotally get you - it's all netflix & doodles for me lately; feels good not draining your wallet every weekend\n\nfound some rad spots by canals where u can chill & sketch with friends - low spend & loads of inspo!\n\n\n\n\n        Inference to